In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, dilation
import glob
import pathlib

In [17]:
rpi = 'h2r3'

# Read input image 
path_to_folder = '../live_bees/' + rpi + '/random_images_of_50/*.jpg'
folder_in = sorted(glob.glob(path_to_folder))
names = [pathlib.Path(x).stem for x in folder_in]
imgs = [cv2.imread(x) for x in folder_in]
n = 0

output_folder = '../a_processed_images/optical_flow/' + rpi + '/' 

In [18]:
masks = []
n = 0
for img in imgs:
    if n % 2 == 0:
        previous_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # previous_gray = cv2.cvtColor(cv2.imread('../live_bees/h2r1/random_images_of_50/hive2_rpi1_240419-211101Z.jpg'), cv2.COLOR_BGR2GRAY)
        n += 1
        continue
    mask = np.zeros_like(img)
    mask[..., 1] = 255
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray = cv2.cvtColor(cv2.imread('../live_bees/h2r1/random_images_of_50/hive2_rpi1_240419-211201Z.jpg'), cv2.COLOR_BGR2GRAY)
    # Calculates dense optical flow by Farneback method 
    flow = cv2.calcOpticalFlowFarneback(previous_gray, gray, flow=None, 
                                        pyr_scale=0.5, # PyrScale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one. default 0.5.
                                        levels=3,      # Number of pyramid layers including the initial image. Levels=1 means that no extra layers are created and only the original images are used. default 5.
                                        winsize=15,    # Averaging window size. Larger values increase the algorithm robustness to image noise and give more chances for fast motion detection, but yield more blurred motion field. default 13.
                                        iterations=3,  # Number of iterations the algorithm does at each pyramid level. default 10.
                                        poly_n=5,      # Size of the pixel neighborhood used to find polynomial expansion in each pixel. Larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field. Typically, PolyN is 5 or 7. default 5.
                                        poly_sigma=1.2,# Standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion. For PolyN=5, you can set PolySigma = 1.1. For PolyN=7, a good value would be PolySigma = 1.5. default 1.1.
                                        flags=0)
        
    # Computes the magnitude and angle of the 2D vectors 
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])   
    # Sets image hue according to the optical flow  
    # direction 
    mask[..., 0] = angle * 180 / np.pi / 2    
    # Sets image value according to the optical flow 
    # magnitude (normalized) 
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    thresh_val = 7.5
    _, thresholded_mask = cv2.threshold(magnitude, thresh_val, 255, cv2.THRESH_BINARY)
    resized_mask = cv2.resize(thresholded_mask, (0, 0), fx=0.5, fy=0.5)
    processed_mask = dilation(resized_mask, disk(15))
    cv2.imwrite(output_folder + names[n-1] + '.jpg', processed_mask)
    # fig, ax = plt.subplots(1, 3, figsize=(15, 15))
    # ax[0].imshow(previous_gray, cmap='gray')
    # ax[0].set_title(names[n-1])
    # ax[1].imshow(thresholded_mask, cmap='gray')
    # ax[1].set_title('Thresholded mask')
    # ax[2].imshow(processed_mask, cmap='gray')
    # ax[2].set_title('Processed mask')
    # plt.show()
    print(processed_mask.shape)
    masks.append(thresholded_mask)
    n += 1


(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)
(1296, 2304)


True